In [1]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
import matplotlib.pyplot as plt
import datetime
import statsmodels.api as sm

In [2]:
def f(tc, t, beta):
    return (abs(tc-t))**beta
def g(tc, t, beta, omega):
    return f(tc, t, beta)*np.cos(omega*np.log(abs(tc-t)))
def h(tc, t, beta, omega):
    return f(tc, t, beta)*np.sin(omega*np.log(abs(tc-t)))

In [25]:

class RLS:
    def __init__(self, X, w):
        '''
        num_vars: number of variables including constant
        lam: forgetting factor, usually very close to 1.
        '''
        # delta controls the initial state.
        self.A = np.linalg.inv(X.T*X)
        self.w = w
        
        # Count of number of observations added
        self.num_obs = len(X)

    def add_obs(self, x, t):
        '''
        Add the observation x with label t.
        x is a column vector as a numpy matrix
        t is a real scalar
        '''            
        z = self.A*x
        alpha = float((1 + x.T*z)**(-1))
        self.w = self.w + (t-alpha*float(x.T*(self.w+t*z)))*z
        self.A -= alpha*z*z.T
        self.num_obs += 1

    #def remove_obs(self, x, t):
    #    z = self.A*x
    #    alpha = float((1 - x.T*z)**(-1))
    #    self.w = self.w - (t-alpha*float(x.T*(self.w+t*z)))*z
    #    self.A += alpha*z*z.T
    #    self.num_obs += 1

In [113]:
tc = 1000
beta = 1
omega = 20
A = 10
B = -4
C1 = 12
C2 = 7
test_size = 1000
# Test function
price = lambda t: A + B*f(tc, t, beta) + C1*g(tc, t, beta, omega) + C2*h(tc, t, beta, omega)
# Gaussian noise to be added to the quadratic signal
noise = np.random.randn(test_size)
# You can play around with other noise (like sinusoidal)
#noise = [np.sin(2*np.pi*i/13) for i in range(test_size)]
y = np.array([price(i) for i in range(test_size)])
noisy_y = np.matrix(y + noise).T



In [163]:
sols = [{} for i in range(test_size)]
for tc in range(900,1100):
     print('tc :' + str(tc))
     for beta in np.linspace(0.1,1,10):
         for omega in np.linspace(0.1,25,10):
            X = np.matrix(np.zeros([test_size,4]))
            time = np.arange(test_size)
            X[:,0] = np.matrix(np.ones(test_size)).T
            X[:,1] = np.matrix(f(tc, t, beta)).T
            X[:,2] = np.matrix(g(tc, t, beta, omega)).T
            X[:,3] = np.matrix(h(tc, t, beta, omega)).T
            if np.linalg.matrix_rank(X[:10].T*X[:10]) != 0:
                w = np.linalg.lstsq(X[:10], noisy_y[:10])[0]
                LS = RLS(X[:10],w)

                for i in range(11,test_size):
                    diff = X[:i]*LS.w - noisy_y[:i]
                    res = np.linalg.norm(diff)**2
                    sols[i][res]={"A":LS.w[0,0],"B":LS.w[1,0],"C1":LS.w[2,0],"C2":LS.w[3,0],"tc":tc,"omega":omega,"beta":beta}
                    x = X[i]
                    LS.add_obs(x.T,noisy_y[i,0])



tc :900


LinAlgError: Singular matrix

In [165]:
np.linalg.matrix_rank(X[:10].T*X[:10]) != 0

True

In [153]:
for tc in range(900,1100):
     #print('tc :' + str(tc))
     for beta in np.linspace(0.1,1,10):
         for omega in np.linspace(0.1,25,10):
            if np.linalg.matrix_rank(X)  != 0:
                print('yes')

In [164]:
np.linalg.inv(X[:10].T*X[:10])

LinAlgError: Singular matrix

In [161]:
X[:10]

matrix([[1.        , 1.99526231, 1.53785268, 1.27125168],
        [1.        , 1.9950627 , 1.53782599, 1.27097065],
        [1.        , 1.9948629 , 1.53779926, 1.27068936],
        [1.        , 1.99466293, 1.53777247, 1.27040782],
        [1.        , 1.99446277, 1.53774562, 1.27012603],
        [1.        , 1.99426243, 1.53771873, 1.26984399],
        [1.        , 1.99406191, 1.53769178, 1.2695617 ],
        [1.        , 1.99386121, 1.53766478, 1.26927915],
        [1.        , 1.99366033, 1.53763772, 1.26899635],
        [1.        , 1.99345926, 1.53761062, 1.2687133 ]])